In [1]:


import resppol.rpol



In [2]:
datei='/home/michael/resppol/resppol/tmp/butanolsybyl.mol2'


In [3]:
test=resppol.rpol.Molecule(datei)

test._bonds
test._atoms


In [4]:
import openforcefield.topology as off
test2=off.Molecule.from_smiles("c1ccccc1")
test.add_conformer_from_mol2(datei)
test.conformers






In [5]:
espfile='/home/michael/resppol/resppol/tmp/butanol/conf0/molecule0.gesp'
test.conformers[0].add_polESP(espfile, "vector")
#test.conformers[0].add_baseESP(espfile)


In [6]:
from __future__ import print_function
from openeye import oechem

mol = oechem.OEGraphMol()
oechem.OESmilesToMol(mol, "c1cocc1")

print("atoms")
for atom in mol.GetAtoms():
    print(atom.GetAtomicNum())
print("bonds")
for bond in mol.GetBonds():
    print(bond.GetOrder())

atoms
6
6
8
6
6
bonds
1
2
1
1
2


In [7]:
from openforcefield.typing.engines.smirnoff.forcefield import ForceField
from openforcefield.utils import get_data_filename
import os
os.getcwd()
print("mytest2")

mytest2


In [15]:

from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField

# Create a simple molecule from SMILES and turn it into a topology.
molecule = Molecule.from_file(datei)
topology = Topology.from_molecules([molecule])

# Let's label using the smirnoff99Frosst force field
forcefield = ForceField('../tmp/BCCPOL.offxml')
# Run the molecule labeling
print(forcefield.get_handler('vdW').parameters)
molecule_force_list = forcefield.label_molecules(topology)




# Print out a formatted description of the parameters applied to this molecule
for mol_idx, mol_forces in enumerate(molecule_force_list):
    print(f'Forces for molecule {mol_idx}')
    for force_tag, force_dict in mol_forces.items():
        print(f"\n{force_tag}:")
        for (atom_indices, parameter) in force_dict.items():
            atomstr=''
            for idx in atom_indices:
                atomstr += '%3s' % idx
            print("atoms: %s  parameter_id: %s  smirks %s" % (atomstr, parameter.id, parameter.smirks) )

[<vdWType with smirks: [#1:1]  epsilon: 0.0157 kcal/mol  id: n1  sigma: 1.069078461768407 A  >, <vdWType with smirks: [#1:1]-[#6]  epsilon: 0.0157 kcal/mol  id: n2  sigma: 2.649532787749369 A  >, <vdWType with smirks: [#6:1]  epsilon: 0.21 kcal/mol  id: n3  sigma: 3.3996695084235347 A  >, <vdWType with smirks: [#6X4:1]  epsilon: 0.1094 kcal/mol  id: n4  sigma: 3.3996695084235347 A  >, <vdWType with smirks: [#8:1]  epsilon: 0.21 kcal/mol  id: n5  sigma: 2.959921901149463 A  >, <vdWType with smirks: [#7:1]  epsilon: 0.17 kcal/mol  id: n6  sigma: 3.249998523775958 A  >, <vdWType with smirks: [#16:1]  epsilon: 0.25 kcal/mol  id: n7  sigma: 3.563594872561357 A  >, <vdWType with smirks: [#15:1]  epsilon: 0.2 kcal/mol  id: n8  sigma: 3.741774616189425 A  >, <vdWType with smirks: [#9:1]  epsilon: 0.061 kcal/mol  id: n9  sigma: 3.1181455134911875 A  >, <vdWType with smirks: [#17:1]  epsilon: 0.265 kcal/mol  id: n10  sigma: 3.4709414058747616 A  >, <vdWType with smirks: [#35:1]  epsilon: 0.32 kc

In [9]:
import numpy as np
np.array(molecule.conformers[0]._value)[0]


array([-1.86500001e+00, -1.51999995e-01,  1.00000005e-03])

In [10]:
print(test.chemical_eq_atoms)

[[4, 6], [5, 6], [7, 8], [9, 10]]


In [11]:
for i, properties in enumerate(molecule_force_list[0]['Bonds'].items()):
            atom_indices, parameter=properties
            print(atom_indices)

(0, 1)
(0, 4)
(0, 5)
(0, 6)
(1, 2)
(1, 7)
(1, 8)
(2, 3)
(2, 9)
(2, 10)
(3, 11)


In [12]:

molecule_force_list[0]
for i, properties in enumerate(molecule_force_list[0]['vdW'].items()):
            atom_index, parameter=properties
            print(atom_index)

(0,)
(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)
(11,)


In [13]:
from __future__ import print_function
from openeye import oechem

def find_eq_atoms(f1):
    """
    Finds all equivalent atoms in a molecule. 
    :return
    Array of pairs of equivalent atoms. 
    
    Parameters:
    ----------
        f1: string
            Path to mol2 file
    
    :TODO Include rdkit support for this function
    """
    
    #Initialize Molecules
    mol1 = oechem.OEGraphMol()
    mol2 = oechem.OEGraphMol()

    # Open Input File Stream
    ifs1 = oechem.oemolistream(f1)
    ifs2 = oechem.oemolistream(f1)

    # Read from IFS to molecules
    oechem.OEReadMol2File(ifs1,mol1)
    oechem.OEReadMol2File(ifs2,mol2)

    qmol = oechem.OEQMol()
    
    # build OEQMol from OEGRAPHMOLECULE
    oechem.OEBuildMDLQueryExpressions(qmol,mol2)
    ss2=oechem.OESubSearch(qmol)
    oechem.OEPrepareSearch(mol1, ss2)
    
    # Store the equivalent atoms
    eq_atoms=[[]for i in range(mol1.NumAtoms())]
    
    # Goes through all matches and compares the atom indeces. 
    for count, match in enumerate(ss2.Match(mol1)):
        for ma in match.GetAtoms():
            # if 2 atoms are not the same atoms
            if ma.pattern.GetIdx() != ma.target.GetIdx():
                    # and the pair is not stored yet
                    if ma.target.GetIdx() not in eq_atoms[ma.pattern.GetIdx()]:
                        # save it to the array
                        eq_atoms[ma.pattern.GetIdx()].append(ma.target.GetIdx())
    
    # goes through the array and returns pairs of equivalent atoms
    sorted_eq_atoms=[]                
    for i,ele in enumerate(eq_atoms):
        if ele!=[]:
            for element in ele:
                # Making sure we have not already covered this pair of atoms
                if [element,i] not in sorted_eq_atoms:
                    sorted_eq_atoms.append([i,element])
            
    
    return(sorted_eq_atoms)
match_atoms(datei)

NameError: name 'match_atoms' is not defined